# Introduction

Python의 Model I/O에 대한 학습함

# FewShotPromptTemplate


In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

# 예시를 주고 그것을 기반으로 답변을 할 수있게 함

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


C:\Users\jhl22\AppData\Local\Temp\ipykernel_7556\56602841.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [2]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}") 

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Human: {question}\nAI:{answer}')

In [3]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "Spain"})

AI: I know this:
        Capital: Madrid
        Language: Spanish
        Food: Paella and Tapas
        Currency: Euro

AIMessage(content='AI: I know this:\n        Capital: Madrid\n        Language: Spanish\n        Food: Paella and Tapas\n        Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--e8b98c65-a662-466d-b1e9-bba6f22cfee2-0')

# FewShotChatMessagePromptTemplate

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [ ]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "what do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
print(example_prompt)
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

examples=[{'country': 'France', 'answer': '\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        '}, {'country': 'Italy', 'answer': '\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        '}, {'country': 'Greece', 'answer': '\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        '}] input_variables=[] input_types={} partial_variables={} example_prompt=ChatPromptTemplate(input_variables=['answer', 'country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='what do you know about {country}?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['answer'], input_types={}, 

In [7]:
chain = final_prompt | chat
chain.invoke(
    {
        "country" : "Korea"
     }
)


        I know this:
        Divided into two countries: North Korea (capital: Pyongyang) and South Korea (capital: Seoul).
        Languages: Korean
        Food: Kimchi and Bibimbap
        Currency: North Korean Won (North) and South Korean Won (South)

AIMessage(content='\n        I know this:\n        Divided into two countries: North Korea (capital: Pyongyang) and South Korea (capital: Seoul).\n        Languages: Korean\n        Food: Kimchi and Bibimbap\n        Currency: North Korean Won (North) and South Korean Won (South)', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--f68a2d55-4e22-4fb1-bee5-a7a3c4267c1e-0')

# LengthBaseExampleSelector

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI (
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],

)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [9]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80
)

In [10]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

# prompt.format(country="Brazil")

In [11]:
chain = prompt | chat

chain.invoke({"country": "Spain"})

AI: Here is what I know about Spain:

- **Capital**: Madrid
- **Language**: Spanish (Castilian)
- **Food**: Tapas, Paella, Jamón Ibérico
- **Currency**: Euro
- **Culture**: Known for its vibrant festivals, flamenco music and dance, and rich history.
- **Geography**: Located in Southwestern Europe, bordered by France, Portugal, and the Mediterranean Sea.

AIMessage(content='AI: Here is what I know about Spain:\n\n- **Capital**: Madrid\n- **Language**: Spanish (Castilian)\n- **Food**: Tapas, Paella, Jamón Ibérico\n- **Currency**: Euro\n- **Culture**: Known for its vibrant festivals, flamenco music and dance, and rich history.\n- **Geography**: Located in Southwestern Europe, bordered by France, Portugal, and the Mediterranean Sea.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--77f10f77-da4c-4a1a-8c49-fc1263838eb1-0')

# RandomBaseExampleSelector

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [ ]:
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

In [ ]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples
)

In [ ]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Greece?\nAI:\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

# Serialization and Composition

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

prompt = load_prompt("./prompt.json")

prompt.format(country="Germmany")

'What is the capital of Germmany'

In [ ]:
chain = prompt | chat

chain.invoke({
    "country": "Germmany"
})

The capital of Germany is Berlin.

AIMessage(content='The capital of Germany is Berlin.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--a0b8bcd3-9354-4fc0-a167-4f2c86314289-0')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

In [ ]:
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
    """
)

example = PromptTemplate.from_template(
    """
    this is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

AIMessage(content="Arrrr, matey! Me favorite grub be a hearty plate o' roasted sea turtle and a side o' salty fish stew! Nothin' like a good feast after a long day o' plunderin' the high seas! Arg arg!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 79, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None}, id='run--ff34a930-8b1b-4058-aa41-871cab52112f-0')

# Caching

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(False)


chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

chat.predict("What is the capital of Japan.")

'The capital of Japan is Tokyo.'

In [ ]:
chat.predict("What is the capital of Japan")

'The capital of Japan is Tokyo.'

In [ ]:
chat.predict("What is the capital of Japan")

'The capital of Japan is Tokyo.'

# Serialization

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug

set_debug(False)

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

with get_openai_callback() as usage:
    chat.predict("What is the recipe for soju")
    print(usage)
    print(usage.total_cost) # 전체 비용
    print(usage.total_tokens) # 토큰 총합
    print(usage.prompt_tokens) # 프롬프트에 사용된 토큰 수
    print(usage.completion_tokens) # 모델이 쓴 토큰

Error in OpenAICallbackHandler.on_llm_end callback: KeyError('total_tokens')


Tokens Used: 0
	Prompt Tokens: 0
		Prompt Tokens Cached: 0
	Completion Tokens: 0
		Reasoning Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
0.0
0
0
0
